# Connect ot SQL Server with pyodbc

References:

https://towardsdatascience.com/sql-server-with-python-679b8dba69fa

https://datatofish.com/how-to-connect-python-to-sql-server-using-pyodbc/

https://pypi.org/project/pyodbc/

https://github.com/mkleehammer/pyodbc/wiki/Cursor

### Step 1: Instal pyodbc library throught conda or pip in command line
To install this library you may need admin right

```python
conda install -c anaconda pyodbc
```
or 
```python
pip instal pyodbc
```


### Step 2: Get the server name
Get the server name that you want to conect to

```sql
SELECT @@SERVERNAME
```

### Step 3: Get the database name
Get the database name that you want to conect to


### Step 4: Get the table or tables names from database
Get the table or tables names that you want to conect to


### Step 5: Connect to SQL Server with Python
Here is the example of code how you can connect to sql server with python

If we are not accessing the database via a trusted connection, we will need to enter the username and password that we would usually use to access the server via SQL Server Management Studio (SSMS).

For example, if our username is <i>JoeBloggs</i>, and our password is <i>Password123</i>, we should immediately change our password.

In [7]:
import pandas as pd
import pyodbc 

# Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns', None)

conn_str = pyodbc.connect('Driver={SQL Server};'
                          #'Driver={SQL Server Native Client 11.0};'
                          'Server=dmreportingdev;'
                          'Database=Soarian;'
                          #'UID=JoeBloggs;'
                          #'PWD=Password123;'
                          'Trusted_Connection=yes;')

# The Cursor object represents a database cursor, which is typically used to manage the context of a fetch operation.
# Database cursors map to ODBC HSTMTs. Cursors created from the same connection are not isolated, i.e. any changes
# done to the database by one cursor are immediately visible by the other cursors. 
# Note, cursors do not manage database transactions, transactions are committed and rolled-back from the connection.

cursor = conn_str.cursor()
cursor.execute('SELECT TOP 10 * FROM [Soarian].[dbo].[M_DIM_CDM]')

for row in cursor:
    print(row)


('3235611', 'CARBIDOPA/LEVODOPA C', 11708, 'Yes', 1.0, 'IP', 80160)
('3235629', 'ESCITALOPRAM 10MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235651', 'LEVOTHYROXINE 0.025MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235652', 'LOSARTAN 25MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235657', 'METHIMAZOLE 5MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235674', 'PRAVASTATIN 40MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235681', 'PYRIDOSTIGMINE 60MG', 11708, 'Yes', 1.0, 'IP', 80160)
('3235699', 'ROSUVASTATIN 10MG TA', 11708, 'Yes', 1.0, 'IP', 80160)
('3235702', 'SIMVASTATIN 10MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)
('3235705', 'SIMVASTATIN 5MG TAB', 11708, 'Yes', 1.0, 'IP', 80160)


In [13]:
# pyodbc extension
x = '3288018'
cursor.execute('SELECT TOP 10 * FROM [Soarian].[dbo].[M_DIM_CDM] WHERE CDM_ID = ?', x)

for row in cursor:
    print(row)

('3288018', 'SODIUM CHLOR .9% 100ML', 11708, 'Yes', 1.0, 'IP', 80160)
('3288018', 'SODIUM CHLOR .9% 100ML', 11708, 'Yes', 1.0, 'OP', 81160)


In [10]:
# Get data from server using pandas pd.read_sql_query or pd.read_sql
df_data = pd.read_sql('SELECT TOP 10 * FROM [Soarian].[dbo].[M_DIM_CDM]', conn_str)
print(type(df_data))
df_data

<class 'pandas.core.frame.DataFrame'>


,CDM_ID,CDM_NAME,COST_CENTER_ID,KEY_STATISTIC,STATISTICAL_WEIGHT,ACCOUNT_TYPE,ACCOUNT_ID
0,3235611,CARBIDOPA/LEVODOPA C,11708,Yes,1.0,IP,80160
1,3235629,ESCITALOPRAM 10MG TAB,11708,Yes,1.0,IP,80160
2,3235651,LEVOTHYROXINE 0.025MG TAB,11708,Yes,1.0,IP,80160
3,3235652,LOSARTAN 25MG TAB,11708,Yes,1.0,IP,80160
4,3235657,METHIMAZOLE 5MG TAB,11708,Yes,1.0,IP,80160
5,3235674,PRAVASTATIN 40MG TAB,11708,Yes,1.0,IP,80160
6,3235681,PYRIDOSTIGMINE 60MG,11708,Yes,1.0,IP,80160
7,3235699,ROSUVASTATIN 10MG TA,11708,Yes,1.0,IP,80160
8,3235702,SIMVASTATIN 10MG TAB,11708,Yes,1.0,IP,80160
9,3235705,SIMVASTATIN 5MG TAB,11708,Yes,1.0,IP,80160


In [17]:
# Get data from server using pandas and parameters 
df_data = pd.read_sql('SELECT TOP 10 * FROM [Soarian].[dbo].[M_DIM_CDM] WHERE CDM_ID = ?', conn_str, params = [x])
print(type(df_data))
df_data

<class 'pandas.core.frame.DataFrame'>


,CDM_ID,CDM_NAME,COST_CENTER_ID,KEY_STATISTIC,STATISTICAL_WEIGHT,ACCOUNT_TYPE,ACCOUNT_ID
0,3288018,SODIUM CHLOR .9% 100ML,11708,Yes,1.0,IP,80160
1,3288018,SODIUM CHLOR .9% 100ML,11708,Yes,1.0,OP,81160


In [18]:
# Get data from server using pandas and join tables 
df_data = pd.read_sql(
    '''SELECT TOP (10)  cdm.CDM_ID
          , cdm.CDM_NAME
          , cdm.COST_CENTER_ID
          , cdm.KEY_STATISTIC
          , cdm.STATISTICAL_WEIGHT
          , cdm.ACCOUNT_TYPE
          , cdm.ACCOUNT_ID
          , a.ACCOUNT_NAME
          , a.ACCOUNT_PARENT
      FROM [Soarian].[dbo].[M_DIM_CDM] cdm
      left join [Soarian].[dbo].[M_DIM_ACCOUNT] a
        on cdm.ACCOUNT_ID = a.ACCOUNT_ID
    ''', conn_str)
print(type(df_data))
df_data



<class 'pandas.core.frame.DataFrame'>


,CDM_ID,CDM_NAME,COST_CENTER_ID,KEY_STATISTIC,STATISTICAL_WEIGHT,ACCOUNT_TYPE,ACCOUNT_ID,ACCOUNT_NAME,ACCOUNT_PARENT
0,3235611,CARBIDOPA/LEVODOPA C,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
1,3235629,ESCITALOPRAM 10MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
2,3235651,LEVOTHYROXINE 0.025MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
3,3235652,LOSARTAN 25MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
4,3235657,METHIMAZOLE 5MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
5,3235674,PRAVASTATIN 40MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
6,3235681,PYRIDOSTIGMINE 60MG,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
7,3235699,ROSUVASTATIN 10MG TA,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
8,3235702,SIMVASTATIN 10MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses
9,3235705,SIMVASTATIN 5MG TAB,11708,Yes,1.0,IP,80160,80160 - IP Unit Doses,Unit Doses


## Change Data in SQL

When we execute queries in SQL, these changes are kept in a temporarily existing space, they are not made directly to the data.

To make these changes permanent, we must commit them.

Lets concatenate the <i>firstName</i> and <i>lastName</i> columns, to create a <i>fullName</i> column.

In [ ]:
cursor = cnxn.cursor()
# first alter the table, adding a column
cursor1.execute("ALTER TABLE customer " +
               "ADD fullName VARCHAR(20)")
# now update that column to contain firstName + lastName
cursor1.execute("UPDATE customer " +
               "SET fullName = firstName + " " + lastName")

In [ ]:
# At this point, fullName does not exist in our database. We must commit these changes to make them permanent:
cnxn.commit()